In [ ]:
import torch
import gc
import logging

# del session.logger
# del session.epoch_logger
# del session
# gc.collect()
# torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v2 import TrainingSessionV2, load_training_session
from config import TrainingConfigV2
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    "armeini2022": {
        "testing_subjects": [],
        "testing_tasks": [0, 1],
    },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=True,
    # # Sensor layout settings
    # layout_dim=2,
    # layout_proj=True,
    # layout_scaling="minmax",
    # # Merger with spatial attn
    # merger=False,
    # merger_emb_type=None,
    # merger_emb_dim=0,
    # merger_channels=0,
    # merger_dropout=0.0,  # Float
    # merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.1,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=15,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV2(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    use_adalora=True,
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(654 * 3),  # 5% total steps
    adalora_tfinal=(654 * 8),  # 50-80% total steps
    adalora_deltaT=(654 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(654 * 50),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=50,
    steps_per_epoch=450, # 654,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives = {
        "clip_loss": 0.6,
        "mse_loss": 0.4
    },
    latent_alignment_objectives= {
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.6,
        "mmd_loss": 0.0
    },
    decode_timestamps=True,
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256
config.steps_per_epoch = 654 # 654
config.epochs = 50

config.brain_encoder_config.in_channels = 269
config.decode_timestamps = True

# Sensor layout settings
config.brain_encoder_config.layout_dim=3
config.brain_encoder_config.layout_proj=False
config.brain_encoder_config.layout_scaling="midpoint"
# Merger with spatial attn
config.brain_encoder_config.merger=True
config.brain_encoder_config.merger_emb_type='mlp'
config.brain_encoder_config.merger_emb_dim=1024
config.brain_encoder_config.merger_channels=269
config.brain_encoder_config.merger_dropout=0.1  # Float
config.brain_encoder_config.merger_conditional=None
config.brain_encoder_config.conditions={
        "study": [],
        # "subject": [],
}
config.brain_encoder_config.conditional_layers=True
config.brain_encoder_config.conditional_layers_dim='input'  # input or hidden_dim

# config.brain_clipping = 20

# config.brain_encoder_config.hidden_dim = 1024
# config.brain_encoder_config.initial_linear = 1024

session = TrainingSessionV2(
    config=config,
    studies={study: "audiotext" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase3/combining/channel_merger_269_mlp_1024_midpoint_3D_dataset_layer",
    clear_cache=False,
    cache_name="/home/ubuntu/cache",
    download_studies=True,
)


# session = load_training_session(
#     save_path="saves/phase3/objectives/baseline_gwilliams_latent_loss_no_latent_alignment/epoch_39",
#     studies={"gwilliams2023": "audiotext"},
#     data_path="data",
#     cache_name="/home/ubuntu/cache",
# )

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=800,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 10, max_cache_size=800
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audiotext
Loading Armeini2022 with batch type audiotext
Data partitioned on studies ['gwilliams2023', 'armeini2022'].
Train: 159, Unseen Task: 51, Unseen Subject: 12, Unseen Both: 4.

Conditional layer study initialized with 3 conditions
RNNEncoder initialized as conformer with 4 layers, 256 d_model, 4 nhead
	Embedding: sinusoidal, params: 6075392
SimpleConv initialized with 10274922 parameters, cond: ['study']
Merger True, merger channels 269
ConvBlocks: 4, hidden_dim: 256, params 2626048
Using torch.bfloat16
Found 64 target modules for AdaLora: ['model.encoder.layers.0.self_attn.k_proj', 'model.encoder.layers.0.self_attn.v_proj', 'model.encoder.layers.0.self_attn.q_proj', 'model.encoder.layers.0.self_attn.out_proj', 'model.encoder.layers.0.fc1', 'model.encoder.layers.0.fc2', 'model.encoder.layers.1.self_attn.k_proj', 'model.encoder.layers.1.self_attn.v_proj', 'model.encoder.layers.1.self_attn.q_proj', 'model.encoder.layers.1.self_attn.out_proj', 

2025-04-02 14:22:52,348	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 159/159 [19:19<00:00,  7.30s/it]


Testing done in 6.00m.
Epoch 1 done in 25.38m. 0.16m/recording.


New best epoch 1 with CER 1.0154 and BLEU 0.0098.
Mel Loss: 6.1651, Clip Loss: 9.5189, MSE: 1.1344
Mel accuracy: 0.0046, Top 5: 0.0228, Top 10: 0.0452


Training Epoch 2: 100%|██████████| 159/159 [19:12<00:00,  7.25s/it]


Testing done in 3.15m.
Epoch 2 done in 22.37m. 0.14m/recording.


New best epoch 2 with CER 0.9708 and BLEU 0.0007.
Mel Loss: 5.8681, Clip Loss: 9.5191, MSE: 0.3916
Mel accuracy: 0.0046, Top 5: 0.0224, Top 10: 0.0452


...

Training Epoch 19: 100%|██████████| 159/159 [19:19<00:00,  7.29s/it]


Testing done in 4.06m.
Epoch 19 done in 23.39m. 0.15m/recording.


Training Epoch 20: 100%|██████████| 159/159 [19:19<00:00,  7.29s/it]


Testing done in 3.75m.
Epoch 20 done in 23.09m. 0.15m/recording.


Training Epoch 21:   0%|          | 0/159 [00:00<?, ?it/s]